In [2]:
from functions_py import *
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [3]:
plt.rcParams.update({
    "image.origin": "lower",
    "image.aspect": 1,
    #"text.usetex": True,
    "grid.alpha": .5,
    "axes.linewidth":2,
    "lines.linewidth" : 1,
    "font.size":    15.0,
    "xaxis.labellocation": 'right',  # alignment of the xaxis label: {left, right, center}
    "yaxis.labellocation": 'top',  # alignment of the yaxis label: {bottom, top, center}
    "xtick.top":           True ,  # draw ticks on the top side
    "xtick.major.size":    8    ,# major tick size in points
    "xtick.minor.size":    4      ,# minor tick size in points
    "xtick.direction":     'in',
    "xtick.minor.visible": True,
    "ytick.right":           True ,  # draw ticks on the top side
    "ytick.major.size":    8    ,# major tick size in points
    "ytick.minor.size":    4      ,# minor tick size in points
    "ytick.direction":     'in',
    "ytick.minor.visible": True,
    "ytick.major.width":   2   , # major tick width in points
    "ytick.minor.width":   1 ,
    "xtick.major.width":   2   , # major tick width in points
    "xtick.minor.width":   1 ,
    "legend.framealpha": 0 ,
    "legend.loc": 'best',

})

In [9]:
dataADU1x10=['dataADUcalib/RunADU_1x10_1.npy',
             'dataADUcalib/RunADU_1x10_2.npy',
             'dataADUcalib/RunADU_1x10_3.npy']
dataADU1x2=['dataADUcalib/RunADU_1x2_1.npy',
            'dataADUcalib/RunADU_1x2_2.npy',
            'dataADUcalib/RunADU_1x2_3.npy']
dataADU1x4=['RunADU_1x4_1.npy',
            'RunADU_1x4_2.npy']
dataADU2x1=['RunADU_2x1_1.npy',
            'RunADU_2x1_2.npy']
dataADU2x2=['RunADU_2x2_1.npy',
            'RunADU_2x2_2.npy']

data_set=[dataADU1x10,dataADU1x2,dataADU1x4,dataADU2x1,dataADU2x2]
data_set_name=['dataADU1x10','dataADU1x2','dataADU1x4','dataADU2x1','dataADU2x2']

In [10]:
for i in [3,4]:
    
    lista=[]
    for file in data_set[i]:
        lista.append(np.load(file))
    np.save(data_set_name[i]+'.npy',np.concatenate(lista,axis=1))
    print(data_set_name[i]+' done! ')

dataADU2x1 done! 
dataADU2x2 done! 


In [ ]:
def GlobalCalib(RunData_adu, RunGain_list, RunNoise_list ,csv_file,extensions=4):
    if extensions!=1:
        for j in extensions:
            global_hist, xb = np.histogram(RunData_adu[j], bins = np.arange(-250, RunGain_list[j]*100+250, 1) ) 
            x = (xb[1:]+xb[:-1])/2

            global_sigma=RunNoise_list[j]*RunGain_list[j]
            peaks, prop = find_peaks(global_hist, distance=4*global_sigma, height=10)
            trunc=abs(np.diff(peaks)-np.median(np.diff(peaks)))/np.median(np.diff(peaks))
            pk=0
            electron_list=[0] #electron number correspondig to each peak 
            for t in trunc.round():
                elec_pk=pk+1+t
                pk=elec_pk
                electron_list.append(elec_pk)
            amp=list(prop['peak_heights']) # initial paramter to curve_fit
            med=list(x[peaks])  # initial paramter to curve_fit

            #Final lists to calibration fit linear
            means_fit=[]
            electrons_fit=[]
            err_means_fit=[]
            out_peaks=csv_file
            with open(out_peaks, "w") as o:
            o.write("#peak, mean, mean_err, sigma, sigma_err, amp, amp_err\n")
            for k in range(len(med)):
                y,xb=np.histogram(RunData_adu,bins=np.arange(med[k]-global_sigma*3,med[k]+global_sigma*3,1))
                x=(xb[1:]+xb[:-1])/2
                try:
                    popt,pcov=curve_fit(gauss,x,y,p0=[med[k],global_sigma,amp[k]])
                    means_fit.append(popt[0])
                    err_means_fit.append(np.sqrt(np.diag(pcov))[0])
                    electrons_fit.append(electron_list[k])
                    perr = np.sqrt(np.diag(pcov))
                    with open(out_peaks, "a") as o:
                        o.write( f"{electron_list[k]}, {popt[0]}, {perr[0]}, {abs(popt[1])}, {perr[1]}, {popt[2]}, {perr[2]}\n" )
                except (RuntimeError,OptimizeWarning,RuntimeWarning):
                    continue
            try:
                popt_g, pcov_g = curve_fit(ax, electrons_fit, means_fit,sigma=err_means_fit, absolute_sigma=True)

            except (RuntimeError,OptimizeWarning,RuntimeWarning):
                print(f"Error - global gain fit failed")
                sys.exit()

            perr=np.sqrt(np.diag(pcov_g))


            print("")
            print("Global Gain a : ",popt_g[0],"+-",perr[0]," ADU/e-")
            print("Global Gain b : ",popt_g[1],"+-",perr[1]," ADU")
            return (popt_g[0],popt_g[1]),len(electrons_fit)

In [36]:
for path in glob.glob('/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/*1.csv'):
    print(path)
    data=pd.read_csv(path,na_values=[' nan',' inf',-1000])
    data=data.dropna(how='any')
    RunGain_list=[np.median(data[data['CHID']==i]['Gain'].values) for i in range(4)]
    RunNoise_list =[np.median(data[data['CHID']==i]['Noise'].values) for i in range(4)]
    rundata_path=path.replace('/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_',
                              '/home/zilvespedro/work/MICROCHIP/MANA/dataADU').replace('_1.csv','.npy')
    print(RunNoise_list)

/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_1x10_1.csv
[0.21202125190770266, 0.1955687886984654, 0.19098674372081034, 0.18972967916248684]
/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_1x2_1.csv
[0.2116075999946698, 0.1956338482169269, 0.1910119036880306, 0.1897318390680469]
/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_1x4_1.csv
[0.21204752012843645, 0.1956216643360677, 0.1911767997342964, 0.18985113943864695]
/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_2x1_1.csv
[0.2111948780839982, 0.196901358174851, 0.19573119277093326, 0.18944595665932795]
/home/zilvespedro/work/MICROCHIP/MANA/AnaParamsCSV/AnaParams_2x2_1.csv
[0.2106189338747109, 0.1966793264835053, 0.1943233548979894, 0.188464662993293]


In [47]:
peaks=25
RunGain_list=[782.6446956497007, 791.4238071687431, 795.2713582541396, 795.1852236417219]
RunNoise_list=[0.21202125190770266, 0.1955687886984654, 0.19098674372081034, 0.18972967916248684]
for path in glob.glob('dataADU1x10.npy'):
    print(path)
    run_data=np.load(path)
    for i in range(4):
        global_hist, xb = np.histogram(run_data[i], bins = np.arange(-150, 790*10+150, 1) ) 
        x = (xb[1:]+xb[:-1])/2
        peaks, prop = find_peaks(global_hist, distance=4*RunNoise_list[i]*RunGain_list[i], height=10)
        trunc=abs(np.diff(peaks)-np.median(np.diff(peaks)))/np.median(np.diff(peaks))
        pk=0
        electron_list=[0] #electron number correspondig to each peak 
        for t in trunc.round():
            elec_pk=pk+1+t
            pk=elec_pk
            electron_list.append(elec_pk)
        print(electron_list)
#         plt.figure(figsize=(20,5))
#         plt.plot(x,global_hist)
#         plt.plot(x[peaks],global_hist[peaks],'x')
#         plt.yscale('log')
# #         plt.xlim(-250, 790*peaks+250)
#     #     plt.ylim(10,1e5)
#         plt.show()

dataADU1x10.npy
[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
[0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
